In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.gold_research_action (
  action_code STRING,
  action_label STRING,
  action_category STRING,        -- lifecycle | evidence | family | identity | narrative
  default_effort INT             -- lower = less effort (1 = low, 2 = medium, 3 = high)
);


In [0]:
%sql
INSERT INTO genealogy.gold_research_action VALUES
-- Evidence & identity
('CLUSTER_SOURCES', 'Find corroborating sources', 'evidence', 2),
('REVIEW_EXISTING_SOURCES', 'Review existing sources closely', 'evidence', 1),
('RESOLVE_NAME_VARIANTS', 'Resolve name variants / aliases', 'identity', 2),

-- Lifecycle
('SEARCH_BIRTH_RECORDS', 'Search for birth/baptism records', 'lifecycle', 1),
('SEARCH_DEATH_RECORDS', 'Search for death/burial records', 'lifecycle', 1),
('SEARCH_MARRIAGE_RECORDS', 'Search for marriage records', 'lifecycle', 1),
('SEARCH_CENSUS', 'Search census / population records', 'lifecycle', 1),

-- Family structure
('TRACE_PARENTS', 'Identify or verify parents', 'family', 2),
('TRACE_CHILDREN_FORWARD', 'Trace children forward', 'family', 2),
('VERIFY_FAMILY_EVENTS', 'Source family events (marriage, children)', 'family', 1),

-- Contextual / narrative
('INVESTIGATE_MIGRATION', 'Investigate migration and movement', 'narrative', 2),
('INVESTIGATE_MILITARY', 'Investigate military service', 'narrative', 3);


In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.gold_research_signal_action (
  signal_code STRING,
  action_code STRING,
  action_weight INT        -- relative importance of this action for this signal (higher value is higher priority)
);


In [0]:
%sql
INSERT INTO genealogy.gold_research_signal_action VALUES
-- Evidence quality
('SIGNAL_LOW_EVIDENCE_DENSITY', 'CLUSTER_SOURCES', 3),
('SIGNAL_SINGLE_SOURCE_DEPENDENCE', 'REVIEW_EXISTING_SOURCES', 3),
('SIGNAL_SINGLE_SOURCE_DEPENDENCE', 'CLUSTER_SOURCES', 2),
('SIGNAL_UNSOURCED_FAMILY_EVENTS', 'VERIFY_FAMILY_EVENTS', 3),

-- Identity quality
('SIGNAL_INCOMPLETE_NAME', 'SEARCH_MARRIAGE_RECORDS', 3),
('SIGNAL_IMPRECISE_DATES', 'REVIEW_EXISTING_SOURCES', 3),

-- Lifecycle gaps
('SIGNAL_NO_BIRTH_RECORDED', 'SEARCH_BIRTH_RECORDS', 3),
('SIGNAL_NO_DEATH_RECORDED', 'SEARCH_DEATH_RECORDS', 3),
('SIGNAL_NO_MARRIAGES', 'SEARCH_MARRIAGE_RECORDS', 3),
('SIGNAL_NO_CHILDREN', 'SEARCH_CENSUS', 3),

-- Family structure
('SIGNAL_MISSING_PARENT', 'TRACE_PARENTS', 3),
('SIGNAL_HAS_CHILDREN', 'TRACE_CHILDREN_FORWARD', 3),
('SIGNAL_MULTIPLE_SPOUSES', 'VERIFY_FAMILY_EVENTS', 3),

-- Context & narrative
('SIGNAL_MIGRANT', 'INVESTIGATE_MIGRATION', 3),
('SIGNAL_MILITARY', 'INVESTIGATE_MILITARY', 3),
('SIGNAL_POSSIBLE_WWI', 'INVESTIGATE_MILITARY', 3),
('SIGNAL_POSSIBLE_WWII', 'INVESTIGATE_MILITARY', 3),

-- Life story flags
('SIGNAL_YOUNG_DEATH', 'REVIEW_EXISTING_SOURCES', 3),
('SIGNAL_HAS_MARRIAGE', 'VERIFY_FAMILY_EVENTS', 3);


In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.ref_signal_weights (
  signal_code STRING,
  category STRING,          -- evidence | completeness
  intent STRING,            -- integrity | narrative
  base_score INT,           -- raw contribution before category weighting
  reason_label STRING,      -- short human-readable reason
  rationale STRING          -- reasoning for given weight for this intent
);


In [0]:
%sql
INSERT OVERWRITE genealogy.ref_signal_weights
VALUES
  -- =====================================================
  -- Evidence fragility signals
  -- =====================================================
  ('SIGNAL_LOW_EVIDENCE_DENSITY',      'evidence',     'integrity',   30, 'low source density', NULL),
  ('SIGNAL_SINGLE_SOURCE_DEPENDENCE',  'evidence',     'integrity',   25, 'single-source reliance', NULL),
  ('SIGNAL_UNSOURCED_FAMILY_EVENTS',   'evidence',     'integrity',   25, 'unsourced family events', NULL),
  ('SIGNAL_IMPRECISE_DATES',           'evidence',     'integrity',   10, 'imprecise dates', 'suggests original documents not consulted'),
  ('SIGNAL_INCOMPLETE_NAME',           'evidence',     'integrity',   10, 'incomplete name', 'suggests exhaustive searches not done'),

  -- =====================================================
  -- Structural completeness signals
  -- =====================================================
  ('SIGNAL_NO_BIRTH_RECORDED',          'completeness',     'integrity',    30, 'birth not recorded', NULL),
  ('SIGNAL_MISSING_PARENT',             'completeness',     'integrity',    25, 'missing parent', NULL),
  ('SIGNAL_NO_DEATH_RECORDED',           'completeness',     'integrity',    20, 'death not recorded', NULL),
  ('SIGNAL_NO_MARRIAGES',                'completeness',     'integrity',    15, 'no marriage recorded', NULL),
  ('SIGNAL_NO_CHILDREN',                 'completeness',     'integrity',    10, 'no children recorded', NULL),

  -- =====================================================
  -- Life texture signals
  -- =====================================================
    ('SIGNAL_MIGRANT',          'texture',     'narrative',    30, 'evidence of international migration', NULL),
    ('SIGNAL_MILITARY',          'texture',     'narrative',    25, 'evidence of military service', NULL),
    ('SIGNAL_POSSIBLE_WWI',     'texture',     'narrative',     15, 'possible evidence of WWI service', NULL),
    ('SIGNAL_POSSIBLE_WWII',    'texture',     'narrative',     15, 'possible evidence of WWII service', NULL),
    ('SIGNAL_YOUNG_DEATH',      'texture',     'narrative',     20, 'young death', NULL),

  -- =====================================================
  -- Family drama signals
  -- =====================================================
    ('SIGNAL_HAS_MARRIAGE',          'family',     'narrative',     20, 'has at least one marriage', NULL),
    ('SIGNAL_HAS_CHILDREN',          'family',     'narrative',     30, 'has children', NULL),
    ('SIGNAL_MULTIPLE_SPOUSES',      'family',     'narrative',     40, 'has multiple spouses', NULL);




In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.ref_intent_category_weights (
  intent STRING,
  category STRING,
  weight DOUBLE
);


In [0]:
%sql
INSERT OVERWRITE genealogy.ref_intent_category_weights
VALUES
  -- =====================================================
  -- Tree integrity categories
  -- =====================================================
  ('integrity',      'evidence',          0.55),
  ('integrity',      'completeness',      0.35),
  ('narrative',      'texture',           0.45),
  ('narrative',      'family',            0.30)  ;